In [2]:
what_to_do = {'0. An end-to-end Scikit-Learn workflow',
 '1. Getting the data ready',
 '2. Choose the right estimator/algorithm for our problems',
 '3. Fit the model/algorithm and use it to make predictions on our data',
 '4. Evaluating a model',
 '5. Improve a model',
 '6. Save and load a trained model',
 '7. Putting it all together!'}

In [3]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

2025-11-02 19:25:44.753908: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-02 19:25:44.933525: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-02 19:25:48.933755: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
car_sales  = pd.read_csv("https://raw.githubusercontent.com/mrdbourke/zero-to-mastery-ml/master/data/car-sales-extended.csv")
car_sales

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431,4,15323
1,BMW,Blue,192714,5,19943
2,Honda,White,84714,4,28343
3,Toyota,White,154365,4,13434
4,Nissan,Blue,181577,3,14043
...,...,...,...,...,...
995,Toyota,Black,35820,4,32042
996,Nissan,White,155144,3,5716
997,Nissan,Blue,66604,4,31570
998,Honda,White,215883,4,4001


In [12]:
X = car_sales.drop("Price", axis=1)
Y = car_sales["Price"]

In [9]:
categorical_cols = ["Make", "Colour"]
numeric_cols = ["Odometer (KM)", "Doors"]

In [13]:
from tensorflow.keras.layers import StringLookup, Normalization

categorical_cols = ["Make", "Colour"]
numeric_cols = ["Odometer (KM)", "Doors"]

# String lookup layers
make_lookup = StringLookup(output_mode="int")
color_lookup = StringLookup(output_mode="int")

make_lookup.adapt(X["Make"])
color_lookup.adapt(X["Colour"])

# Normalization layer — FIX
normalizer = Normalization()
normalizer.adapt(X[numeric_cols].to_numpy())  # use X not x


In [22]:
def preprocess_features(x):
    make = tf.cast(make_lookup(x["Make"]), tf.float32)
    color = tf.cast(color_lookup(x["Colour"]), tf.float32)

    # Combine numeric columns in a 1D tensor
    numeric = tf.stack([
        tf.cast(x["Odometer (KM)"], tf.float32),
        tf.cast(x["Doors"], tf.float32)
    ])
    
    # Normalize numeric features
    numeric = normalizer(tf.expand_dims(numeric, axis=0))
    numeric = tf.squeeze(numeric)

    # Combine all features into one tensor
    return tf.concat([make[tf.newaxis], color[tf.newaxis], numeric], axis=0)


In [23]:
full_data = tf.data.Dataset.from_tensor_slices((dict(X), Y.values))
full_data = full_data.shuffle(buffer_size=len(car_sales), seed=42)


In [24]:
full_data = full_data.map(lambda x, y: (preprocess_features(x), tf.cast(y, tf.float32)))


In [25]:
train_size = int(0.8 * len(car_sales))
train_data = full_data.take(train_size).batch(32)
test_data = full_data.skip(train_size).batch(32)


In [26]:
pd.DataFrame(train_data)

2025-11-02 19:49:47.585753: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,0,1
0,"((tf.Tensor(3.0, shape=(), dtype=float32), tf....","(tf.Tensor(14986.0, shape=(), dtype=float32), ..."
1,"((tf.Tensor(1.0, shape=(), dtype=float32), tf....","(tf.Tensor(24834.0, shape=(), dtype=float32), ..."
2,"((tf.Tensor(4.0, shape=(), dtype=float32), tf....","(tf.Tensor(50868.0, shape=(), dtype=float32), ..."
3,"((tf.Tensor(1.0, shape=(), dtype=float32), tf....","(tf.Tensor(15960.0, shape=(), dtype=float32), ..."
4,"((tf.Tensor(1.0, shape=(), dtype=float32), tf....","(tf.Tensor(14698.0, shape=(), dtype=float32), ..."
5,"((tf.Tensor(2.0, shape=(), dtype=float32), tf....","(tf.Tensor(14202.0, shape=(), dtype=float32), ..."
6,"((tf.Tensor(1.0, shape=(), dtype=float32), tf....","(tf.Tensor(17258.0, shape=(), dtype=float32), ..."
7,"((tf.Tensor(1.0, shape=(), dtype=float32), tf....","(tf.Tensor(11036.0, shape=(), dtype=float32), ..."
8,"((tf.Tensor(1.0, shape=(), dtype=float32), tf....","(tf.Tensor(6121.0, shape=(), dtype=float32), t..."
9,"((tf.Tensor(2.0, shape=(), dtype=float32), tf....","(tf.Tensor(14431.0, shape=(), dtype=float32), ..."


In [28]:
pd.DataFrame(test_data)

,0,1
0,"((tf.Tensor(2.0, shape=(), dtype=float32), tf....","(tf.Tensor(15297.0, shape=(), dtype=float32), ..."
1,"((tf.Tensor(1.0, shape=(), dtype=float32), tf....","(tf.Tensor(30191.0, shape=(), dtype=float32), ..."
2,"((tf.Tensor(1.0, shape=(), dtype=float32), tf....","(tf.Tensor(9918.0, shape=(), dtype=float32), t..."
3,"((tf.Tensor(3.0, shape=(), dtype=float32), tf....","(tf.Tensor(16730.0, shape=(), dtype=float32), ..."
4,"((tf.Tensor(1.0, shape=(), dtype=float32), tf....","(tf.Tensor(8834.0, shape=(), dtype=float32), t..."
5,"((tf.Tensor(2.0, shape=(), dtype=float32), tf....","(tf.Tensor(26078.0, shape=(), dtype=float32), ..."
6,"((tf.Tensor(1.0, shape=(), dtype=float32), tf....","(tf.Tensor(5285.0, shape=(), dtype=float32), t..."


In [31]:
# lets refit the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.mae,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["mae"])

model.fit(train_data, epochs=100)

Epoch 1/100


2025-11-02 19:53:56.423761: I external/local_xla/xla/service/service.cc:163] XLA service 0x760d8806aa00 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-02 19:53:56.423823: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-11-02 19:53:56.510051: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-02 19:53:57.037095: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91400
2025-11-02 19:53:57.090639: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-02 19:53:57.

22/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 16077.2967 - mae: 16077.2967

I0000 00:00:1762093446.385094    8273 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


25/25 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - loss: 16114.7373 - mae: 16114.7373
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 16195.0479 - mae: 16195.0479
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 16180.9150 - mae: 16180.9150
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 15963.0283 - mae: 15963.0283
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 15493.5146 - mae: 15493.5146
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 15604.2695 - mae: 15604.2695
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15113.9873 - mae: 15113.9873
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 14272.2695 - mae: 14272.2695
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 13472.8984 - mae: 13472.8984
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 12511.9404 - mae: 12511.9404
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 10823.7773 - mae: 10823.7773
Epoch 12/100
25/25 ━━━━━━━━━━━